In [1]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model  import Ridge,Lasso,RidgeCV, LassoCV, ElasticNet, ElasticNetCV, LogisticRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import seaborn as sns
import pickle
import os

In [2]:
pwd()

'D:\\iNeuron\\FSDS\\Machine_Learning'

In [3]:
def get_filepaths(directory):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.

In [4]:
lst=get_filepaths("AReM")

In [25]:
lst

['AReM\\bendingType.pdf',
 'AReM\\sensorsPlacement.pdf',
 'AReM\\bending1\\dataset1.csv',
 'AReM\\bending1\\dataset2.csv',
 'AReM\\bending1\\dataset3.csv',
 'AReM\\bending1\\dataset4.csv',
 'AReM\\bending1\\dataset5.csv',
 'AReM\\bending1\\dataset6.csv',
 'AReM\\bending1\\dataset7.csv',
 'AReM\\bending2\\dataset1.csv',
 'AReM\\bending2\\dataset2.csv',
 'AReM\\bending2\\dataset3.csv',
 'AReM\\bending2\\dataset4.csv',
 'AReM\\bending2\\dataset5.csv',
 'AReM\\bending2\\dataset6.csv',
 'AReM\\cycling\\dataset1.csv',
 'AReM\\cycling\\dataset10.csv',
 'AReM\\cycling\\dataset11.csv',
 'AReM\\cycling\\dataset12.csv',
 'AReM\\cycling\\dataset13.csv',
 'AReM\\cycling\\dataset14.csv',
 'AReM\\cycling\\dataset15.csv',
 'AReM\\cycling\\dataset2.csv',
 'AReM\\cycling\\dataset3.csv',
 'AReM\\cycling\\dataset4.csv',
 'AReM\\cycling\\dataset5.csv',
 'AReM\\cycling\\dataset6.csv',
 'AReM\\cycling\\dataset7.csv',
 'AReM\\cycling\\dataset8.csv',
 'AReM\\cycling\\dataset9.csv',
 'AReM\\lying\\dataset1.csv'

In [40]:
df=pd.DataFrame(columns = ["time",'avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23','label'], index = None)

try:    
    for i in lst:
        current_link=(os.getcwd()+'\\'+i)  

        if current_link.endswith('.csv'):
            print(current_link)
            df1=pd.read_csv(current_link,skiprows=4)
            df1['label'] = current_link.split("\\")[-2]
            print(df1)
            df.append(df1)
        else:
            print(current_link)
except:
    print('Error')
print(df)

    

D:\iNeuron\FSDS\Machine_Learning\AReM\bendingType.pdf
D:\iNeuron\FSDS\Machine_Learning\AReM\sensorsPlacement.pdf
D:\iNeuron\FSDS\Machine_Learning\AReM\bending1\dataset1.csv
     # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                  0      39.25       0.43      22.75       0.43      33.75   
1                250      39.25       0.43      23.00       0.00      33.00   
2                500      39.25       0.43      23.25       0.43      33.00   
3                750      39.50       0.50      23.00       0.71      33.00   
4               1000      39.50       0.50      24.00       0.00      33.00   
..               ...        ...        ...        ...        ...        ...   
475           118750      43.33       0.47      25.00       0.00      30.00   
476           119000      43.50       0.50      25.50       0.50      30.00   
477           119250      43.50       0.50      24.75       0.43      30.00   
478           119500      43.50      

D:\iNeuron\FSDS\Machine_Learning\AReM\bending2\dataset1.csv
     # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                  0      23.75       0.43      24.00       0.00      24.67   
1                250      22.67       0.94      24.00       0.00      24.67   
2                500      21.33       0.47      24.00       0.00      24.75   
3                750      21.00       0.00      24.00       0.00      25.25   
4               1000      23.00       0.71      23.75       0.43      24.75   
..               ...        ...        ...        ...        ...        ...   
475           118750      27.50       0.50      22.75       1.30      21.00   
476           119000      27.00       0.00      21.00       0.00      20.75   
477           119250      27.00       0.00      21.00       0.00      20.50   
478           119500      27.25       0.43      23.50       0.87      20.75   
479           119750      27.00       0.00      24.00       0.00      1

D:\iNeuron\FSDS\Machine_Learning\AReM\cycling\dataset12.csv
     # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                  0      33.50       2.87      27.00       1.87      14.75   
1                250      30.50       4.50      20.67       4.03      17.00   
2                500      34.25       3.03      15.75       4.92      17.25   
3                750      42.00       2.55      16.00       2.74      19.75   
4               1000      42.00       2.83      19.50       2.06      13.75   
..               ...        ...        ...        ...        ...        ...   
475           118750      34.75       2.17      17.75       1.92      19.00   
476           119000      39.00       3.00      14.00       3.27      15.00   
477           119250      40.25       4.26      16.33       3.68      14.33   
478           119500      32.00       5.74      17.75       4.44      18.75   
479           119750      34.00       9.27      22.50       1.50      1

In [5]:
print(os.getcwd()+lst[2])

D:\iNeuron\FSDS\Machine_LearningAReM\bending1\dataset1.csv


In [25]:
df

""


In [15]:
print(str(os.getcwd()+ str(lst[2])))

D:\iNeuron\FSDS\Machine_LearningAReM\bending1\dataset1.csv


In [18]:
link.split("\\")[-2]

'bending1'

In [10]:
link = str(os.getcwd()+'\\'+ str(lst[2]))

In [12]:
df = pd.read_csv(link,skiprows=4)

In [13]:
df

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,39.25,0.43,22.75,0.43,33.75,1.3
1,250,39.25,0.43,23.00,0.00,33.00,0.0
2,500,39.25,0.43,23.25,0.43,33.00,0.0
3,750,39.50,0.50,23.00,0.71,33.00,0.0
4,1000,39.50,0.50,24.00,0.00,33.00,0.0
...,...,...,...,...,...,...,...
475,118750,43.33,0.47,25.00,0.00,30.00,0.0
476,119000,43.50,0.50,25.50,0.50,30.00,0.0
477,119250,43.50,0.50,24.75,0.43,30.00,0.0
478,119500,43.50,0.50,24.33,0.47,30.00,0.0


In [19]:
df['label'] = link.split("\\")[-2]

In [20]:
df

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,label
0,0,39.25,0.43,22.75,0.43,33.75,1.3,bending1
1,250,39.25,0.43,23.00,0.00,33.00,0.0,bending1
2,500,39.25,0.43,23.25,0.43,33.00,0.0,bending1
3,750,39.50,0.50,23.00,0.71,33.00,0.0,bending1
4,1000,39.50,0.50,24.00,0.00,33.00,0.0,bending1
...,...,...,...,...,...,...,...,...
475,118750,43.33,0.47,25.00,0.00,30.00,0.0,bending1
476,119000,43.50,0.50,25.50,0.50,30.00,0.0,bending1
477,119250,43.50,0.50,24.75,0.43,30.00,0.0,bending1
478,119500,43.50,0.50,24.33,0.47,30.00,0.0,bending1
